In [ ]:
import os
import numpy as np
import pandas as pd
import torch

In [ ]:
bboxes_folder_path = os.listdir("../../../../data/add_disk1/panos/abacus/bboxes_dave_sr/")
print(bboxes_folder_path)

df = pd.read_csv("/home/panos/zsoc/data/FSC147_384_V2/annotations/abacus_v8.csv")

for folder in bboxes_folder_path:
    candidates = os.listdir(os.path.join("../../../../data/add_disk1/panos/abacus/bboxes_dave_sr/", folder))
    print(candidates)

In [ ]:
import os
import torch
import clip
from PIL import Image

def load_clip_model(device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Load the CLIP model and preprocess pipeline.
    """
    model, preprocess = clip.load("ViT-B/32", device=device)
    return model, preprocess

def calculate_similarity(image_embeddings, text_embedding):
    """
    Compute cosine similarity between image embeddings and a text embedding.
    """
    # Normalize embeddings to unit vectors
    image_embeddings = image_embeddings / image_embeddings.norm(dim=-1, keepdim=True)
    text_embedding = text_embedding / text_embedding.norm(dim=-1, keepdim=True)

    # Cosine similarity
    similarity = torch.matmul(image_embeddings, text_embedding.T)
    return similarity.squeeze()

def select_exemplars_from_crops(model, preprocess, crop_folder, object_name, num_exemplars=10, device="cuda:1"):
    """
    Select exemplars based on CLIP embeddings from cropped images.

    Parameters:
    - crop_folder (str): Path to the folder containing cropped images.
    - object_name (str): The name of the object to match (e.g., "elephant").
    - num_exemplars (int): Number of exemplars to select.
    - device (str): Device to run the model on ("cuda" or "cpu").

    Returns:
    - List of selected image file paths and their similarity scores.
    """
    # Load CLIP model and preprocessing pipeline

    # Load cropped images and extract embeddings
    crop_files = [os.path.join(crop_folder, f) for f in os.listdir(crop_folder)]
    image_embeddings = []
    for crop_file in crop_files:
        image = Image.open(crop_file).convert("RGB")
        processed_image = preprocess(image).unsqueeze(0).to(device)
        with torch.no_grad():
            image_embedding = model.encode_image(processed_image)
        image_embeddings.append(image_embedding)

    # Stack image embeddings into a single tensor
    image_embeddings = torch.cat(image_embeddings, dim=0)

    # Extract embedding for the object name
    with torch.no_grad():
        text_embedding = model.encode_text(clip.tokenize([f"{object_name}"]).to(device))

    # Compute similarities
    similarities = calculate_similarity(image_embeddings, text_embedding)

    # Select top image files based on similarity
    top_indices = similarities.argsort(descending=True)[:num_exemplars]
    selected_files = [crop_files[i] for i in top_indices]

    return selected_files, similarities[top_indices].cpu().numpy()

In [ ]:
from tqdm import tqdm
import shutil

model, preprocess = load_clip_model("cuda:1")

for i, folder in tqdm(enumerate(bboxes_folder_path)):
    candidates_folder = os.path.join("../../../../data/add_disk1/panos/abacus/bboxes_dave_sr/", folder)
    object_name = df.loc[df["filename"] == (folder + ".jpg"), "prompt_notation"].values[0]
    
    selected_files, scores = select_exemplars_from_crops(model, preprocess, candidates_folder, object_name)
    print("Selected Files:", selected_files)
    print("Similarity Scores:", scores)

    for exemplar in selected_files[:5]:
        # Destination directory
        destination_dir = os.path.join(os.path.join(os.path.join("../../../../data/add_disk1/panos/abacus/bboxes_dave_sr/", folder), "0_prototypes"))
        # Ensure the destination directory exists
        os.makedirs(destination_dir, exist_ok=True)

        # Copy the file
        shutil.copy(exemplar, destination_dir)
        #print(f"File copied to {destination_dir}")


In [ ]:
crop_folder = "../../../../data/add_disk1/panos/abacus/bboxes_dave_sr/7565"  # Path to the folder containing cropped images
object_name = "a sticky note"        # The object class name to match

selected_files, scores = select_exemplars_from_crops(crop_folder, object_name)
print("Selected Files:", selected_files)
print("Similarity Scores:", scores)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# List of image paths
image_paths = selected_files

# Create subplots
fig, axes = plt.subplots(1, len(image_paths), figsize=(15, 5))

for ax, img_path in zip(axes, image_paths):
    image = Image.open(img_path)
    ax.imshow(image)
    ax.axis("off")  # Remove axes
    #ax.set_title(img_path.split("/")[-1])  # Optional: Set image name as title

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# List of image paths
folder = 7565
image_paths = [f'../../../../data/add_disk1/panos/abacus/bboxes_sr/{folder}/prototypes/prototype_object_1_sr.png', f'../../../../data/add_disk1/panos/abacus/bboxes_sr/{folder}/prototypes/prototype_object_2_sr.png', f'../../../../data/add_disk1/panos/abacus/bboxes_sr/{folder}/prototypes/prototype_object_3_sr.png']

# Create subplots
fig, axes = plt.subplots(1, len(image_paths), figsize=(15, 5))

for ax, img_path in zip(axes, image_paths):
    image = Image.open(img_path)
    ax.imshow(image)
    ax.axis("off")  # Remove axes
    ax.set_title(img_path.split("/")[-1])  # Optional: Set image name as title

plt.tight_layout()
plt.show()